# Download YouTube Video's Audio

In [5]:
! pip install pytube -q

In [6]:
from pytube import YouTube

In [7]:
#VIDEO_URL = "https://www.youtube.com/watch?v=hWLf6JFbZoo" #obama

In [8]:
VIDEO_URL = 'https://www.youtube.com/watch?v=5MuIMqhT8DM&pp=ygUIdGVkIHRhbGs%3D' #batman

In [9]:
#VIDEO_URL = 'https://youtu.be/qNJRGHk7sN8'

In [10]:
yt = YouTube(VIDEO_URL)

In [11]:
yt.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'ytaudio.mp4')

'/content/ytaudio.mp4'

In [12]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# English ASR with HuggingSound

In [13]:
!pip install huggingsound -q

In [14]:
from huggingsound import SpeechRecognitionModel


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [15]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = 0)


INFO:huggingsound.speech_recognition.model:Loading model...


OUT OF MEMORY (OOM) error

# Audio Chunking

In [16]:
import librosa

In [17]:
input_file = '/content/ytaudio.wav'

In [18]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [19]:
import soundfile as sf


In [20]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [21]:
i

38

# Audio Transcription / ASR / Speech to Text

In [22]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [23]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav',
 '/content/9.wav',
 '/content/10.wav',
 '/content/11.wav',
 '/content/12.wav',
 '/content/13.wav',
 '/content/14.wav',
 '/content/15.wav',
 '/content/16.wav',
 '/content/17.wav',
 '/content/18.wav',
 '/content/19.wav',
 '/content/20.wav',
 '/content/21.wav',
 '/content/22.wav',
 '/content/23.wav',
 '/content/24.wav',
 '/content/25.wav',
 '/content/26.wav',
 '/content/27.wav',
 '/content/28.wav',
 '/content/29.wav',
 '/content/30.wav',
 '/content/31.wav',
 '/content/32.wav',
 '/content/33.wav',
 '/content/34.wav',
 '/content/35.wav',
 '/content/36.wav',
 '/content/37.wav',
 '/content/38.wav']

In [24]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 39/39 [00:22<00:00,  1.75it/s]


In [25]:
full_transcript = ' '

In [26]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [27]:
len(full_transcript)

14101

In [28]:
full_transcript

" thank you very much well i would like to start with testaclesmen who sleep five hours a night have significantly smaller testacles than those who sleep seven hours or morein addition men who routinely sleep just forty-five hours at night will have a level of testosterone which is that of someone ten years their senorso a lack of sleep will age a man by a decade in terms of that critical aspect of wellnessand we see equivalent imperments in female reproductive health caused by a lack of sleepthis is the best news that i have for you todayfrom this point it may only get worse not only will i tell you about the wonderfully good things that happen when you get asleep but the alarmingly bad things that happen when you don't get enough hour brain and feel bodylet me start with the brain andfunctions of learning in memorybecause whadiscovered over the past ten-orso years is that you need sleep after learning to essentially hit the save button on those new memories so that you don't forgetbu

# Text Summarization

In [29]:
from transformers import pipeline

In [32]:
summarization = pipeline('summarization', model = 'facebook/bart-large-cnn')

In [34]:
# Define the maximum sequence length for the model
max_seq_length = 1024

# Split the text into chunks
chunks = [full_transcript[i:i + max_seq_length] for i in range(0, len(full_transcript), max_seq_length)]

# Initialize a list to store the summaries
summaries = []

# Summarize each chunk
for chunk in chunks:
    summarized_text = summarization(chunk)
    summaries.append(summarized_text[0]['summary_text'])  # Assuming only one summary is generated per chunk

# Combine the summaries into a single summary for the entire text
combined_summary = "\n".join(summaries)

# Print or use the combined summary
print(combined_summary)


Men who sleep five hours a night have significantly smaller testacles than those who sleep seven hours or more. Men who routinely sleep just forty-five hours at night will have a level of testosterone which is that of someone ten years their senor. A lack of sleep will age a man by a decade in terms of that critical aspect of wellness.
Without sleep the memory circuits of the brain essentially become waterlogged as it were and you can't absorb new memories so let me show you the data dhere in this study we decided to test the hypothesis that pulling the old nighter was a good ideas. that you also need sleep before learningand w o actually preper your brain almost like a dry sponge ready to soak up new information.
The hippocampus is a structure that sits on e left and right side of your brain called the hippocampus. It's very good receiving new memory files and then holding onto them. In those people who haa full night of sleep we saw lots of healthy learning-related activity. Yet in p

Text Chunking before Summarization

In [31]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
 # print("Summarized text\n"+out)
  summarized_text.append(out)

#print(summarized_text)

In [32]:
print(summarized_text)

[' A few years ago i felt like i was stuck in a rut so i decided', ' Tens of thousands of people try to write their own fifty-thousand-word novels', " i guarantee you the next thirty days a're going to pass whether you like it or"]
